In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# AWS
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION = os.getenv("AWS_REGION")
AWS_S3_OUTPUT_BUCKET = os.getenv("AWS_S3_OUTPUT_BUCKET")
AWS_S3_INPUT_BUCKET = os.getenv("AWS_S3_INPUT_BUCKET")

In [2]:
import boto3
import os

class Bucket:

  def __init__(self, name:str):
    self.name = name
    self.client = boto3.client(
        's3',
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name=AWS_REGION  # Optional
    )

  def list_objects(self):
    print(f"Listing objects in bucket: {self.name}")
    response = self.client.list_objects_v2(Bucket=self.name)
    return response.get('Contents', [])

  def download_object(self, key, directory_name, return_file_path=False):
    # Create the local directory if it doesn't exist
    os.makedirs(directory_name, exist_ok=True)
    local_file_path = os.path.join(directory_name, key)
    print(f"Downloading object from {self.name}: {key} to {local_file_path}")
    self.client.download_file(self.name, key, local_file_path)
    if return_file_path:
      return local_file_path

  def upload_object(self, local_file_path, key):
    print(f"Uploading object from {local_file_path} to {self.name}: {key}")
    self.client.upload_file(local_file_path, self.name, key)
    print(f"Uploaded object from {local_file_path} to {self.name}: {key}")

  def get_object(self, key):
    print(f"Getting object: {key} from {self.name}")
    response = self.client.get_object(Bucket=self.name, Key=key)
    return response['Body'].read()
